In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from scipy.signal import savgol_filter
import random  
import time
import io
import h5py
import tensorflow as tf
from tensorflow.keras.datasets import mnist     # MNIST dataset is onderdeel van Keras
from tensorflow.keras.models import Sequential  # Het type neuraal netwerk dat we gaan gebruiken
from tensorflow.keras.layers import Dense, Dropout, Activation  # Verschillende type lagen die we gaan gebruiken
from tensorflow.keras.optimizers import SGD, Adam, schedules
from tensorflow.keras import utils
import sys


configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=configuration)
print("done")

done


# Data prep

In [2]:
from PIL import Image

def get_pixels_val(image_path): # bron: https://stackoverflow.com/questions/138250/how-to-read-the-rgb-value-of-a-given-pixel-in-python
    """Get a numpy array of an image so that one can access values[x][y]."""
    image = Image.open(image_path, "r")
    pixel_values = np.array(image)

    # width crop
    pixel_values = pixel_values[54:-43]
    
    # height crop
    pv = list()
    for c, h in enumerate(pixel_values):
        pv.append(pixel_values[c, 35:-36])
    
    return np.array(pv)
print("done")

done


# Create transformed and halved csv file

In [3]:
    
folder = r"D:\FARM_data\Soil_Spectra_s1"
def data_clean(countryfile):  
    # opent de og sat data file

    with open(r"D:\FARM_data\Soil_Spectra\spectra_ "+countryfile+" .csv") as f:
        # maakt csv reader aan
        reader = csv.reader(f)
        # Open 
        with open(folder+"\spectra_ "+countryfile+" .csv", 'w', newline='') as file:
            writer = csv.writer(file)
            for c, row in enumerate(reader):
                if c == 0:
                    writer.writerow(row[:5] + row[205:-200:2])
                else:
                    x = np.array(row[205:-200:2], dtype='float64')
                    reflectance = 10**(-x) # van absorbance naar reflectance
                    writer.writerow(row[:5] + list(reflectance))


In [16]:
range_names = [f'{i-20}-{i}'for i in range(20, 561, 20)]
print(range_names)

['0-20', '20-40', '40-60', '60-80', '80-100', '100-120', '120-140', '140-160', '160-180', '180-200', '200-220', '220-240', '240-260', '260-280', '280-300', '300-320', '320-340', '340-360', '360-380', '380-400', '400-420', '420-440', '440-460', '460-480', '480-500', '500-520', '520-540', '540-560']


In [7]:
co_data_path = r"D:\FARM_data\LUCAS2015_topsoildata_20200323\LUCAS_Topsoil_2015_20200323.csv"

col_list = ["Point_ID", "NUTS_0", "OC"]

df = pd.read_csv(co_data_path, usecols=col_list)
df
# df["OC_state"] = "medium"
# max_value = df["OC"].max()
# df.loc[df['OC'] < (max_value * (1/3)), ['OC_state']] = "bad"
# df.loc[df['OC'] > (max_value * (2/3)), ['OC_state']] = "good"
# print(df['OC'].value_counts())
# print(df['OC'].unique())
# plt.bar(df['OC'].value_counts(),df['OC'].unique())

In [8]:
countries = ["BE"] #DE has problems, ES has memory problems, file size is shit
# "AT", "BE", "NL", "UK","EL", "EE","DK", "CZ", "CY", "BG", "FI", "FR", "HR", "HU","IE", "IT", "LT", "LU", "LV", "MT", "PL", "PT", "RO",
# "SE", "SI", "SK"
for country in countries:
    print(country)
    df_country = df.loc[df['NUTS_0'] == country]
    df_country["spectogram"] = np.nan
    pd.options.mode.chained_assignment = None  #turns an annoying warning off, nothing important
    df_country["spectogram"] = df_country["spectogram"].astype(object)

    data_clean(country)
    with open(folder+"\spectra_ " + country +" .csv") as fi:
        re = csv.reader(fi)
        lst = list()
        start = time.time()
        row1= np.array(next(re)[5:])

        for c, ro in enumerate(re):
            pointid = ro[2]
            if int(pointid) in df_country['Point_ID'].values:
                reflectance = np.array(ro[5:])
                r = savgol_filter(reflectance, 11, 2)
                
                # reflectance spectograms
                f, t, Sxx = signal.spectrogram(r, 1)

                sxx_sdv = np.array([[ (Sxx[i][j] - np.mean(Sxx[i]) ) / np.std(Sxx[i]) for j in range(Sxx.shape[1])] for i in range(Sxx.shape[0])])

                fig, ax = plt.subplots()
                ax.axis('off')
                ax.pcolormesh(t, f, sxx_sdv, shading='gouraud')#, shading='gouraud'

                io_buf = io.BytesIO()
                fig.savefig(io_buf, format='rgba', dpi=72)
                io_buf.seek(0)
                img_arr = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
                                     newshape=(int(fig.bbox.bounds[3]), int(fig.bbox.bounds[2]), -1))
                io_buf.close()
                plt.close()


                array = img_arr[35:-36]
                array = array[:, 54:-43]
                array = np.delete(array[:], -1, -1)
                df_country.loc[df_country['Point_ID'] == int(pointid), ['spectogram']] = [array]



    df_country = df_country.dropna()
    df_country.to_hdf(folder+"\ data.hdf5", country)
    print(time.time() - start)

# TODO: ervoor zorgen dat sneller csv gevuld word (met batches miss werken), ervoor zorgen dat het goed geprint word
# npz = np.load(r"D:\FARM_data\Soil_Spectra_s1\spectra_ BE_array.npz")

BE


C:\Anaconda3\lib\site-packages\pandas\core\generic.py:2431: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['NUTS_0', 'OC_state', 'spectogram'], dtype='object')]

  pytables.to_hdf(


19.010596752166748


In [ ]:
# reread = pd.read_hdf(folder+"\labeled_data.hdf5", key="BE")
# reread

In [ ]:
print(100*17/170%10)

In [ ]:
f.create_dataset('BE', data=lst)

In [ ]:
f.close()

In [ ]:
hf = h5py.File(folder+"\mytestfile.hdf5", 'a')

In [ ]:
if 3980204 in df_country['Point_ID'].values:
    print("true")
else:
    print("false")

In [ ]:
print(20%200)

In [ ]:
# TODO: Labelen van Spectogram arrays + opsplitsen train en test?(kan miss ook met train_test_split), 
# Het opstellen van een model
folder = "D:\FARM_data\Soil_Spectra_s1"
# df_country.to_hdf(folder+"\mytestfile.hdf5", 'BE')
f = h5py.File(folder+'\labeled_data.hdf5', 'r')
reread = pd.read_hdf(folder+"\labeled_data.hdf5", key='BE')
print('done')


In [ ]:
print(reread)

In [ ]:
# for i in f.keys():
#     print(i)

In [ ]:
# for z in reread['spectogram']:
#     print(z.shape)

In [ ]:
# # Bron: https://www.pythoninformer.com/python-libraries/numpy/numpy-and-images/
# img = Image.open('image.jpeg')
# array = np.array(img)
# array = array[35:-36] #54:-43
# array = array[:, 54:-43]
# invimg = Image.fromarray(array)
# invimg.save('TRANSFORM.jpeg')

In [ ]:
print(reread)

In [ ]:
plt.title("Line graph") 
plt.xlabel("X axis") 
plt.ylabel("Y axis") 
plt.plot(row1, r, color ="red") 
plt.show()

In [ ]:
            
#                 print(array)
            #lst.append(array)
#         print(len(lst))
            
# -80 sec (100)
#             plt.savefig(f'image.jpeg')

#             img = Image.open('image.jpeg')
#             print(type(img))
# -3 sec (100)
#             array = np.array(img)
#             array = array[35:-36]
#             array = array[:, 54:-43]
# - 3 sec (100)
#             lst.append(array)
#         np.savez(fil, *lst)